In [1]:
cd("/home/gridsan/aramadhan/Oceananigans.jl/")
pwd()

"/home/gridsan/aramadhan/Oceananigans.jl"

In [3]:
zF = [1, 2, 4, 7, 11]

5-element Array{Int64,1}:
  1
  2
  4
  7
 11

In [2]:
using Pkg
Pkg.activate(".")

"/home/gridsan/aramadhan/Oceananigans.jl/Project.toml"

In [3]:
using Oceananigans, Oceananigans.Operators

┌ Info: Recompiling stale cache file /home/gridsan/aramadhan/.julia/compiled/v1.1/Oceananigans/hU93i.ji for Oceananigans [9e8cae18-63c1-5223-a75c-80ca9d6e9a09]
└ @ Base loading.jl:1184


CUDA-enabled GPU(s) detected:
CuDevice(0): Tesla V100-SXM2-16GB


In [4]:
using Random, Statistics
using FFTW

In [5]:
function ∇²!(grid::RegularCartesianGrid, f, ∇²f)
    for k in 1:grid.Nz, j in 1:grid.Ny, i in 1:grid.Nx
        @inbounds ∇²f[i, j, k] = ∇²(grid, f, i, j, k)
    end
end

∇²! (generic function with 1 method)

In [35]:
Nz, Lz = 8, 1

Rall = rand(MersenneTwister(0), Nz+1)
R = Rall[1:Nz]
Rall[Nz+1] = 0
Rall[1:Nz] = Rall[1:Nz] .- mean(Rall[1:Nz])

R = Rall[1:Nz]

8-element Array{Float64,1}:
  0.4900442230630878 
  0.5767532530121119 
 -0.16903748678063935
 -0.15627443844806   
 -0.05472317558312356
 -0.1301267268724019 
 -0.2913016189822949 
 -0.2653340294086798 

In [36]:
R2 = FFTW.r2r(R, FFTW.REDFT10)

ks = 1:Nz |> Array
kz² = @. (2sin((ks-1)*π/(2Nz)) / (Lz/Nz))^2

R3 = -R2 ./ kz²
R3[1] = 0

R4 = FFTW.r2r(R3, FFTW.REDFT01)
 
@. R4 = R4 / 2Nz

grid = RegularCartesianGrid((1, 1, Nz), (1, 1, Lz))
fbcs = DoublyPeriodicBCs()

ϕ   = CellField(Float64, CPU(), grid)
∇²ϕ = CellField(Float64, CPU(), grid)

data(ϕ) .= real.(reshape(R4, 1, 1, Nz))

fill_halo_regions!(grid, (:T, fbcs, ϕ.data))
∇²!(grid, ϕ, ∇²ϕ)

fill_halo_regions!(grid, (:T, fbcs, ∇²ϕ.data))

data(∇²ϕ) ≈ reshape(R, 1, 1, Nz)

true

In [16]:
using LinearAlgebra

In [17]:
ld = [1, 2, 3]
ud = [4, 5, 6]
d  = [7, 8, 9, 10]
M = Tridiagonal(ld, d, ud)

4×4 Tridiagonal{Int64,Array{Int64,1}}:
 7  4  ⋅   ⋅
 1  8  5   ⋅
 ⋅  2  9   6
 ⋅  ⋅  3  10

In [48]:
Δz = (Lz/Nz) * ones(Nz+1)

9-element Array{Float64,1}:
 0.125
 0.125
 0.125
 0.125
 0.125
 0.125
 0.125
 0.125
 0.125

In [49]:
δ(k) = -(Δz[k-1] + Δz[k+1]) / (Δz[k-1] * Δz[k+1])

δ (generic function with 1 method)

In [73]:
ud = [1/Δz[k] for k in 2:Nz+1]

8-element Array{Float64,1}:
 8.0
 8.0
 8.0
 8.0
 8.0
 8.0
 8.0
 8.0

In [74]:
ld = [1/Δz[k] for k in 1:Nz]

8-element Array{Float64,1}:
 8.0
 8.0
 8.0
 8.0
 8.0
 8.0
 8.0
 8.0

In [75]:
d = [-1/Δz[1], [δ(k) for k in 2:Nz]..., -1/Δz[Nz+1]]

9-element Array{Float64,1}:
  -8.0
 -16.0
 -16.0
 -16.0
 -16.0
 -16.0
 -16.0
 -16.0
  -8.0

In [76]:
M = Tridiagonal(ld, d, ud)

9×9 Tridiagonal{Float64,Array{Float64,1}}:
 -8.0    8.0     ⋅      ⋅      ⋅      ⋅      ⋅      ⋅     ⋅ 
  8.0  -16.0    8.0     ⋅      ⋅      ⋅      ⋅      ⋅     ⋅ 
   ⋅     8.0  -16.0    8.0     ⋅      ⋅      ⋅      ⋅     ⋅ 
   ⋅      ⋅     8.0  -16.0    8.0     ⋅      ⋅      ⋅     ⋅ 
   ⋅      ⋅      ⋅     8.0  -16.0    8.0     ⋅      ⋅     ⋅ 
   ⋅      ⋅      ⋅      ⋅     8.0  -16.0    8.0     ⋅     ⋅ 
   ⋅      ⋅      ⋅      ⋅      ⋅     8.0  -16.0    8.0    ⋅ 
   ⋅      ⋅      ⋅      ⋅      ⋅      ⋅     8.0  -16.0   8.0
   ⋅      ⋅      ⋅      ⋅      ⋅      ⋅      ⋅     8.0  -8.0

In [77]:
g = Δz .* Rall

9-element Array{Float64,1}:
  0.06125552788288598 
  0.07209415662651399 
 -0.021129685847579918
 -0.0195343048060075  
 -0.006840396947890445
 -0.016265840859050237
 -0.03641270237278686 
 -0.03316675367608497 
  0.0                 

In [78]:
M \ g

SingularException: SingularException(9)

In [79]:
cond(Matrix(M))

3.569394875300831e17

In [103]:
# Reference CPU implementation per Numerical Recipes, Press et. al 1992 (§ 2.4)
function tridiag(M::Tridiagonal{T,<:Array}, f::Vector{T})::Vector{T} where T
    N = length(f)
    ϕ = similar(f)
    γ = similar(f)

    β    = M.d[1]
    ϕ[1] = f[1] / β

    for j = 2:N
        γ[j] = M.du[j-1] / β
        β    = M.d[j] - M.dl[j-1] * γ[j]

        # This should only happen on last element of forward pass for problems
        # with zero eigenvalue. In that case the algorithmn is still stable.
        abs(β) < 1.0e-12 && break

        ϕ[j] = (f[j] - M.dl[j-1] * ϕ[j-1]) / β
    end

    for j = 1:N-1
        k = N-j
        ϕ[k] = ϕ[k] - γ[k+1] * ϕ[k+1]
    end

    return ϕ
end

tridiag (generic function with 1 method)

In [108]:
ψ = tridiag(M, g)

9-element Array{Float64,1}:
 -0.07351280144284969  
 -0.06585586045748894  
 -0.049187149893813946 
 -0.03515965006108644  
 -0.023573938329109874 
 -0.012843276215619611 
 -0.0041458442095106286
 -3.469446951953614e-18
  0.0                  

In [109]:
grid = RegularCartesianGrid((1, 1, Nz), (1, 1, Lz))
fbcs = DoublyPeriodicBCs()

ϕ   = CellField(Float64, CPU(), grid)
∇²ϕ = CellField(Float64, CPU(), grid)

data(ϕ) .= real.(reshape(ψ[1:Nz], 1, 1, Nz))

fill_halo_regions!(grid, (:T, fbcs, ϕ.data))
∇²!(grid, ϕ, ∇²ϕ)

fill_halo_regions!(grid, (:T, fbcs, ∇²ϕ.data))

data(∇²ϕ) ≈ reshape(R, 1, 1, Nz)

true

In [110]:
data(∇²ϕ)

1×1×8 view(OffsetArray(::Array{Float64,3}, 0:2, 0:2, 1:8), 1:1, 1:1, 1:8) with eltype Float64:
[:, :, 1] =
 0.49004422306308815

[:, :, 2] =
 0.5767532530121113

[:, :, 3] =
 -0.16903748678063923

[:, :, 4] =
 -0.15627443844805988

[:, :, 5] =
 -0.05472317558312356

[:, :, 6] =
 -0.1301267268724019

[:, :, 7] =
 -0.2913016189822949

[:, :, 8] =
 -0.26533402940868